## Fluídos Conformes

Código para testear la estabilidad alrededor de una solución de equilibrio.

Las ecuaciones son:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k
$$

$$
c^j = C^j(f) = (e, S, c_i)
$$

Lo que hacemos es tomar el jacobiano del flujo,

$\frac{\partial F^i}{\partial f^j}$ donde $f^j$ son las variables no conservativas para las cuales tenemos las expresiones del Flujo. 
A estas las multiplicamos por $[\frac{\partial c^k}{\partial f^j}]^{-1}$.
Tenemos así 

$$
P^{il}{}_j := \frac{\partial F^{il}}{\partial c^j} = \frac{\partial F^{il}}{\partial f^k}\frac{\partial f^k}{\partial c^j}.
$$

Evaluamos todo en el vector $(-1,0,0,0,0)$ o sea estamos en el frame con velocidad cero y todas las variables fuera de equilibrio también cero.

Lo mismo hacemos con la fuente. 
Tenemos así el sistema linearizado en el punto de equilibrio en las variables conservativas. Multiplicamos $P^i{}_j$ por un número complejo cualquiera y para todos ellos los autovalores debieran tener parte real negativa o nula. Pero no es así...

In [4]:
using Symbolics
using LinearAlgebra

┌ Info: Precompiling Symbolics [0c5d862f-8b57-4792-8d23-62f2024744c7]
└ @ Base loading.jl:1317


In [97]:
function Flux(flu,par)
    χ = par
    #flux = zeros(5)
    μ = flu[1]  
    T = (abs(μ))^(-1//2) 
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    γ = (1 - v^2)^(-1//2)
    τ = 2χ₁ * x3 * T / (γ*μ^3) + 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) + 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
#    Q = 10χ₀ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    Q = 10χ₁ * x2 * T / μ^3 + 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)
    #flux[1] = 4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v
    #flux[2] = 4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ
    #flux[3] = χ₁*γ*v*(6*γ^2 - 1)/μ^3/T - 12χ₂*(v*(6γ^2 - 1)*x1 + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4
    #flux[4] = χ₁*γ*(6γ^2*v^2+1)/μ^3/T - 12χ₂*((6γ^2*v+1)*x1 + v*(6γ^2*(1+v^2)+1)*x2 + (2v^2+1)*x3)/μ^4
    #flux[5] = 3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 +3*(1+6γ^2*v^2)*x2 + 3v*x3)/μ^4
    #return -flux[:]
    return [4//3 * ρ*γ^2*v + γ*Q*(1+v^2)+ τ*v; 
            4//3 * ρ*(γ^2*v^2 + 1//4) + 2v*γ*Q + τ;
            χ₁*γ*v*(6*γ^2 - 1)/μ^3/T   - 12χ₂*(v*(6γ^2 - 1)*x1   + (6γ^2*(2v^2 + 1)-1)*x2 + v*(v^2+2)*x3)/μ^4;
            χ₁*γ*(6γ^2*v^2+1)/μ^3/T    - 12χ₂*((6γ^2*v+1)*x1     + v*(6γ^2*(1+v^2)+1)*x2  + (2v^2+1)*x3)/μ^4;
            3χ₁*γ*v*(2γ^2*v^2+1)/μ^3/T - 12χ₂*(v*(6γ^2*v^2+3)*x1 + 3*(1+6γ^2*v^2)*x2      + 3v*x3)/μ^4]
end

Flux (generic function with 1 method)

Chequeamos con los valores de Marcelo. 

In [98]:
χ = [- 1.; - 2.; - 10.]
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
Flu = [5.921317694643806; 6.02302807825841; 3.713391378258412;  4.136735467078638; 3.444079555898864]

Flux(flu,χ) - Flu;

Calculamos el Jacobiano

In [99]:
@variables f[1:5], c[1:5], p[1:3]#, fl[1:5]

JFS = Symbolics.jacobian(Flux(f,p),f);
JF_exp = Symbolics.build_function(JFS, f, p);
JFac = eval(JF_exp[1]);

Definimos algunos valores para testear

In [100]:
@variables x1, x2, x3

flu=[-1.;0;0;0;0];
con=ones(5)
χ=[-1.,-0.5,-5.]

3-element Vector{Float64}:
 -1.0
 -0.5
 -5.0

El Jacobiano en equilibrio.

In [101]:
JFac(flu,p)

5×5 Matrix{Num}:
  0.0    (-8//1)*p₁          0.0    -10.0p₂      0.0
 -4.0p₁           0.0       -2.0p₂    0.0       -2.0p₂
  0.0            -5.0p₂     -0.0    -60.0p₃     -0.0
 -2.5p₂           0.0    -12p₃       -0.0    -12p₃
  0.0            -3.0p₂     -0.0    -36.0p₃     -0.0

Ahora traemos el Jacobiano de la transformación entre las variables fluido/conservativas.

In [102]:
include("inversion_ext.jl");

Para posterior uso calculamos las variables conservativas correspondientes al estado de equilibrio. Esto no me lo esperaba, pensaba que me darían c1=c2=c3=0!

In [175]:
con = F(flu,zeros(5), p);

In [176]:
Jac(flu,con,p)

5×5 Matrix{Num}:
 12.0p₁           0.0       -6.0p₂     0.0        0.0
  0.0    (-8//1)*p₁          0.0    10p₁          0.0
 -7.5p₂           0.0      -36.0p₃    -0.0        0.0
  0.0            15.0p₂     -0.0     -60.0p₃     -0.0
 -7.5p₂           0.0    -12p₃        -0.0    -12p₃

In [177]:
P(flu,c,p) = Jac(flu,c,p) \ JFac(flu,p)

P (generic function with 1 method)

In [179]:
P(flu,c,p);

In [180]:
P(flu,c,p)'

5×5 adjoint(::Matrix{Num}) with eltype Num:
  0.0                                                                                                    …            0.0
 -0.666667                                                                                                            0.166667p₂*(p₃^-1)
  0.0                                                                                                                -0.0
  0.0833333(p₁^-1)*(10p₂ + 6.0p₂*(6.25(p₁^-1)*(p₂^2) - (60.0p₃))*((-36.0p₃ - (3.75(p₁^-1)*(p₂^2)))^-1))     (-1//12)*(6.25(p₁^-1)*(p₂^2) - (36.0p₃) - ((-12p₃ - (3.75(p₁^-1)*(p₂^2)))*(6.25(p₁^-1)*(p₂^2) - (60.0p₃))*((-36.0p₃ - (3.75(p₁^-1)*(p₂^2)))^-1)))*(p₃^-1)
  0.0                                                                                                                -0.0

In [108]:
Ratio = P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3]

-0.08333333333333333(p₁^-1)*(6.0p₂*(6.25(p₁^-1)*(p₂^2) - (60.0p₃))*((3.75(p₁^-1)*(p₂^2) - (36.0p₃))^-1) - (10.0p₂))*(3.75(p₁^-1)*(p₂^2) - (36.0p₃))*((6.25(p₁^-1)*(p₂^2) - (60.0p₃))^-1)

In [109]:
#par_f = [-1,-0.5,-10]
#substitute(P(flu,c,p)'[4,1]/P(flu,c,p)'[4,3], (Dict(p => par_f),))


In [110]:
simplify(12*(10p[1]*(-36p[3] - 3.75p[2]^2/p[1]) + 6p[2]*(6.25p[2] - 60p[3])) / (6.25p[2] - 60p[3]) / p[1])

(p₁^-1)*((6.25p₂ - (60p₃))^-1)*(120p₁*(-36p₃ - (3.75(p₁^-1)*(p₂^2))) + 72p₂*(6.25p₂ - (60p₃)))

In [111]:
par_f = [-1,-0.5,-10]
P(flu,c,par_f)'[4,1]/P(flu,c,par_f)'[4,3]

0.0

Miramos primero el caso donde las variables están desacopladas, es decir con $\chi_1=0$.

In [181]:
P(flu,c,[-1,0,-1])

5×5 Matrix{Float64}:
 -0.0  -0.666667  -0.0   -0.0      -0.0
 -0.5   0.0        0.25   0.0       0.25
  0.0   0.0        0.0    1.66667   0.0
  0.0   0.0        0.2    0.0       0.2
  0.0   0.0        0.0    1.33333   0.0

Vemos que se corresponde con la siguiente matriz racional.

In [188]:
M = [0 -2//3 0 0 0;
    -1//2 0 1//4 0 1//4;
    0 0 0 5//3 0;
    0 0 1//5 0 1//5;
    0 0 0 4//3 0]

5×5 Matrix{Rational{Int64}}:
  0//1  -2//3  0//1  0//1  0//1
 -1//2   0//1  1//4  0//1  1//4
  0//1   0//1  0//1  5//3  0//1
  0//1   0//1  1//5  0//1  1//5
  0//1   0//1  0//1  4//3  0//1

In [189]:
P(flu,c,[-1,0,-1]) - M

5×5 Matrix{Float64}:
 -0.0  0.0  -0.0  -0.0          -0.0
  0.0  0.0   0.0   0.0           0.0
  0.0  0.0   0.0  -2.22045e-16   0.0
  0.0  0.0   0.0   0.0           0.0
  0.0  0.0   0.0   0.0           0.0

In [190]:
eigvals(M)

5-element Vector{Float64}:
 -0.7745966692414842
 -0.5773502691896256
  3.0357660829594124e-18
  0.5773502691896258
  0.7745966692414837

In [191]:
1/0.7745966692414837^2 - 5/3

-1.5543122344752192e-15

In [192]:
1/0.5773502691896258^2 - 3

-8.881784197001252e-16

In [193]:
eigvals(P(flu,c,[-1,-0.5,-1]))

5-element Vector{Float64}:
 -0.8303267893222077
 -0.5621119867018832
  5.900956464450273e-18
  0.5621119867018834
  0.8303267893222093

Hacemos lo mismo para la fuente.

In [194]:
function Is(flu, χ, ξ)
    #(χ, ξ) = par 
    μ = flu[1] 
    χ₀= χ[1]
    χ₁= -χ[2] # lo hacemos positivo
    κ = χ₀*ξ[1]/χ₁^2 # OK 
    λ = χ₀*ξ[2]/χ₁^2  # OK
    η = χ₀*ξ[3]/χ₁^2
    T = (abs(μ))^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[2]
    x1 = flu[3]
    x2 = flu[4]
    x3 = flu[5]
    γ = (1. - v^2)^(-1//2)
    #Is[1] = 0.
    #Is[2] = 0.
    #Is[3] = -2//5*a*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ)
    #Is[4] = -2//5*a*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ)
    #Is[5] = -2//5*a*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
    #return Is[:]      #(1 - ℯ^(-5. *t))
    return [0.;
            0.;
            -(3//10*(γ^2-1//4)*x1/(γ*λ)   + 2γ*v*x2/κ      - v^2*x3/(γ*η))/μ^4/T;
            -(3//10*γ*v*x1/λ              + γ*(v^2+1)*x2/κ - v*x3/(γ*η))/μ^4/T;
            -(3//10*(γ^2*v^2+1//4)*x1/λ/γ + 2γ*v*x2/κ      - x3/(γ*η))/μ^4/T;
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*η);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*η);
            #-2//5*(γ^2*v^2+1//4)*T*x1/η/γ - 2γ*v*x2/T/κ + T*x3/(γ*η)
            #-2//5*(γ^2-1//4)*T*x1/(γ*λ) - 2γ*v*x2/T/κ - v^2*T*x3/(γ*λ);
            #-2//5*γ*v*T*x1/λ - γ*(v^2+1)*x2/T/κ - v*T*x3/(γ*λ);
            #-2//5*(γ^2*v^2+1//4)*T*x1/λ/γ - 2γ*v*x2/T/κ - T*x3/(γ*λ)
            ]
end

Is (generic function with 1 method)

La chequeamos con valores de Marcelo.

In [195]:
t=0
flu = [- 5.; 0.5; 2.1; 0.5; 5.1]
par = [-1., -2., -10.]
par_s = [1/15π, 1/1π, 1/1π]
I_c = [0.; 0.; - 0.09488575328013785; - 0.12155655385334033; - 0.12140081195218293]
Is(flu,par,par_s) - I_c;

In [196]:
@variables q[1:3]  
Is(f,p,q);

In [197]:
JSS = Symbolics.jacobian(Is(f,p,q),f);

In [198]:
JS_exp = Symbolics.build_function(JSS, f, p,q);

In [199]:
JSac = eval(JS_exp[1]);

In [200]:
flu=[1.;0;0;0;0];
con=ones(5)
par_f = [-1,-2,-10]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, -0.1/1π]
JSac(flu,par_f,par_s)
eigvals(JSac(flu,par_f,par_s)) #esta no es la matríz correcta para sacarle los autovalores

5-element Vector{Float64}:
 -188.49555921538757
   -2.8274333882308134
    0.0
    0.0
  125.66370614359172

In [201]:
II(flu,c,p,q) = Jac(flu,c,p) \ JSac(flu,p,q)

II (generic function with 1 method)

In [202]:
II(flu,c,p,q);

In [203]:
par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, -0.1/1π]
eigvals(10*im .* P(flu,c,par_f) - 1.0 .*II(flu,c,par_f,par_s))

5-element Vector{ComplexF64}:
   -0.036354831936990496 + 6.94483159742221e-17im
   -0.004149839024724664 - 7.794725217674606im
   -0.004149839024720325 + 7.794725217674601im
 -0.00017955904272781709 + 5.758437692400371im
   -0.000179559042724983 - 5.758437692400375im

In [204]:
eigvals(-II(flu,c,par_f,par_s)) #notemos que esta tiene dos autovalor positivos!

5-element Vector{Float64}:
 -0.06544984694978735
 -0.0
  0.0
  0.0004895988551049027
  0.019946620022792336

Vemos que un par de autovalores tiene parte real positiva. 

In [205]:
eigvecs(II(flu,c,par_f,par_s))

5×5 Matrix{Float64}:
 0.0        0.242536    1.0  0.0  0.0
 0.780869   0.0         0.0  1.0  0.0
 0.0        0.970142    0.0  0.0  0.0
 0.624695   0.0         0.0  0.0  0.0
 0.0       -3.75171e-5  0.0  0.0  1.0

In [206]:
par_f = [-1,-2,-10]
E = eigvecs(10*im .* P(flu,c,par_f) - 0. * II(flu,c,par_f,par_s));

In [207]:
[E[:,1]; E[:,5]]

10-element Vector{ComplexF64}:
  0.8347534056877877 + 0.0im
 -0.4884783787108429 + 0.1107591222771439im
 0.16820651508815548 + 0.07480332624201028im
 0.06892618846789017 + 0.03834262219161344im
   0.092827541786135 + 0.05984266099360827im
  0.8347534056877881 + 0.0im
 -0.4884783787108414 - 0.11075912227714245im
 0.16820651508815684 - 0.07480332624200983im
 0.06892618846789086 - 0.038342622191613385im
 0.09282754178613609 - 0.059842660993607785im

In [208]:
[E[:,2]; E[:,3]]

10-element Vector{ComplexF64}:
     0.8347534056877879 + 0.0im
     0.4884783787108423 + 0.11075912227714238im
    0.16820651508815604 - 0.07480332624200943im
   -0.06892618846789045 + 0.03834262219161309im
    0.09282754178613542 - 0.05984266099360755im
     0.8162604286217225 + 0.0im
 -2.244927941254197e-16 + 5.014786892247826e-16im
     0.3911247887145761 + 2.7755575615628914e-17im
 -1.698266897373134e-17 + 1.4051203440729406e-17im
   -0.42513563990714787 + 2.7755575615628914e-17im

In [209]:
E[:,4]

5-element Vector{ComplexF64}:
  0.8347534056877881 + 0.0im
  0.4884783787108423 - 0.110759122277142im
 0.16820651508815593 + 0.07480332624200926im
 -0.0689261884678904 - 0.038342622191613024im
 0.09282754178613536 + 0.05984266099360734im

In [210]:
#10*im .* P(flu,c,par) - II(flu,c,par_f,par_s)

In [211]:
Ei = inv(E)

5×5 Matrix{ComplexF64}:
   0.317021+0.619127im     …  0.350622-1.08641im
   0.317021-0.619127im        0.350622+1.08641im
 -0.0717131+8.04153e-17im     -1.43426-1.81146e-17im
   0.317021+0.619127im        0.350622-1.08641im
   0.317021-0.619127im        0.350622+1.08641im

In [212]:
Ei[5,:]

5-element Vector{ComplexF64}:
  0.31702074473528674 - 0.6191270079459985im
   -0.363592318253284 + 0.6536076882838987im
 -0.28049761746161805 + 2.4729703084866848im
   1.0502976514020446 + 4.632103282986477im
   0.3506220218270598 + 1.0864088285514342im

In [213]:
Ei[5,:]'*E[:,5]

3.2751579226442118e-15 - 8.913009219568835e-15im

In [214]:
ET = eigvecs(10*im .* P(flu,c,par_f)');

In [215]:
[ET[:,1]; ET[:,5]];

In [216]:
[ET[:,2]; ET[:,3]];

In [217]:
ET[:,4] - ET[1,4]/Ei[4,1] .* Ei[4,:];

In [218]:
1/(Ei[4,1]/Ei[4,3])

-3.34836285560923 - 1.2614582446639144im

In [219]:
1/(Ei[4,1]/Ei[4,5])

-1.1604938271604954 - 1.1605415850907999im

 [-1,-1,-1] -> 16/5, 4  $\frac{4\chi_2}{\chi_1}(4/5,1)$
 [-1,-2,10] -> -16, 20
 [-1,-0.5,-1] -> -64, 80

In [220]:
4χ[2]/χ[1]*[4/5;1]

2-element Vector{Float64}:
 1.6
 2.0

In [221]:
function coc(par_f)
    return 4par_f[3]/par_f[2]*[4/5 ; 1]
end

coc (generic function with 1 method)

In [222]:
coc([-1,-2,-10])

2-element Vector{Float64}:
 16.0
 20.0

In [223]:
coc([-1,-0.5,-10])

2-element Vector{Float64}:
 64.0
 80.0

Alternativamente podemos quedarnos con las expresiones en derivadas del los campos del fluido.
Es decir:

$(\zeta^a, \zeta^{ab})$ -> $f^j = (\mu, v, x_i)$ -> $(-1,0,0,0,0)$

$$
\partial_t c^j = \partial_i F^{ij}(f) + I^j(f) = \frac{\partial F^{ij}}{\partial f^k}\partial_i f^k + I^j(j)
$$

In [224]:
flu=[1.;0;0;0;0];
con=ones(5)
par_f = [-1.,-2.,-10.]
#par_f = [-1,-0.5,-10]
par_s = [-1/15π, -1/1π, -0.1/1π]
PP = 1.0im.* JFac(flu,par_f) + 1.0.*JSac(flu,con,par_s)

5×5 Matrix{ComplexF64}:
 0.0+0.0im  0.0+8.0im        0.0+0.0im        0.0-20.0im        0.0-0.0im
 0.0-4.0im  0.0+0.0im        0.0+4.0im        0.0-0.0im         0.0-4.0im
 0.0+0.0im  0.0-10.0im  0.706858+0.0im        0.0+600.0im       0.0+0.0im
 0.0+5.0im  0.0+0.0im        0.0+120.0im  47.1239+0.0im         0.0+120.0im
 0.0+0.0im  0.0-6.0im   0.235619+0.0im        0.0+360.0im  -31.4159+0.0im

In [243]:
H=[-3/4 0 0 0 0;
   0 -3/2 0 0 0;
   0 0 6/10 0 0;
   0 0 0 -3 0;
   0 0 0 0 1]

5×5 Matrix{Float64}:
 -0.75   0.0  0.0   0.0  0.0
  0.0   -1.5  0.0   0.0  0.0
  0.0    0.0  0.6   0.0  0.0
  0.0    0.0  0.0  -3.0  0.0
  0.0    0.0  0.0   0.0  1.0

In [244]:
H*JFac(flu,p)

5×5 Matrix{Num}:
  0.0      6.0p₁   0.0     -7.5p₂   0.0
 -6.0p₁    0.0     3.0p₂    0.0    -3.0p₂
  0.0      3.0p₂   0.0    -36.0p₃   0.0
  7.5p₂    0.0    36.0p₃    0.0    36.0p₃
  0.0    3p₂       0.0    -36.0p₃   0.0

In [171]:
eigvals(H*(1.0im.* JFac(flu,par_f) - 0.0.*JSac(flu,con,par_s)))

5-element Vector{ComplexF64}:
      -5.876237734640878 + 2.079099117315793e-14im
 -2.2737367544323206e-13 + 508.89441947216665im
                     0.0 + 0.0im
   4.973799150320701e-14 - 508.894419472167im
      5.8762377346408785 + 1.8305797049248772e-14im

In [227]:
H*JSac(flu,con,p)

5×5 Matrix{Num}:
 0.0  0.0   0.0            0.0             0.0
 0.0  0.0   0.0            0.0             0.0
 0.0  0.0  -0.135(p₂^-1)   0.0             0.0
 0.0  0.0   0.0           -3.0(p₁^-1)      0.0
 0.0  0.0  -0.075(p₂^-1)   0.0         p₃^-1

In [228]:
1/0.135

7.4074074074074066

In [230]:
200/27

7.407407407407407